In [0]:
# %cd data
# !pwd
# !gunzip GoogleNews-vectors-negative300.bin
# %cd ..

In [0]:
#INPUTS

#repoPath should start with the path 
#'/content/gdrive/My Drive/' and go to the folder where the git repo is, and should end with the name of the repo:
repoPath = '/content/gdrive/My Drive/TestingGitCloneIntoGoogleDrive/textSimilarityUsingWordMoversDistance'
userEmail = 'sanmesh1@gmail.com'
userName = 'sanmesh1'

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
! pip install ipdb -q
import ipdb 

#place the line of code below from whichever line you want to start debugging
#type n for step over and s for step into
# ipdb.set_trace()

In [0]:
import os
os.chdir(repoPath)
#%cd gdrive/My Drive/project_folder/TextSimilarityUsingWord2Vec

In [0]:
from time import time
start_nb = time()

In [0]:
# Import and download stopwords from NLTK.
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.

# Remove stopwords.
stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
start = time()
import os
from gensim.models import Word2Vec

%cd data/
!ls
import os.path
from os import path
if path.exists("GoogleNews-vectors-negative300.bin") == False:
  !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
  !gunzip GoogleNews-vectors-negative300.bin
else:
  print("datasetAlreadyExists")
%cd ..

from gensim.models.keyedvectors import KeyedVectors
model_path = './data/GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(model_path, binary=True)

print('Cell took %.2f seconds to run.' % (time() - start))

/content/gdrive/My Drive/TestingGitCloneIntoGoogleDrive/textSimilarityUsingWordMoversDistance/data
GoogleNews-vectors-negative300.bin
datasetAlreadyExists
/content/gdrive/My Drive/TestingGitCloneIntoGoogleDrive/textSimilarityUsingWordMoversDistance


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Cell took 75.43 seconds to run.


In [0]:
# Pre-processing a document.

from nltk import word_tokenize
download('punkt')  # Download data for tokenizer.

def preprocess(doc):
    doc = doc.lower()  # Lower the text.
    doc = word_tokenize(doc)  # Split into words.
    doc = [w for w in doc if not w in stop_words]  # Remove stopwords.
    doc = [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    return doc

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
#Create pandas dataframe on dataset
import pandas as pd
#https://data.world/crowdflower/ecommerce-search-relevance
df = pd.read_csv('https://query.data.world/s/iccclhgzbyedtv54p5lresglnun3qz', encoding = "ISO-8859-1")

In [0]:
#create source document and target document we will be comparing scores for
numDataPoints = 6000

sourceTitle = df['product_title'][2584]
source_doc = df['product_description'][2584]

targetTitles = df['product_title'][:numDataPoints].tolist()
target_docs = df['product_description'][:numDataPoints].tolist()

In [0]:
# def calculateWmdSimilarity(source_doc, otherDocs):
#   source_doc_preproc = preprocess(source_doc)
#   otherDocs
#   preprocess(sent)
test = preprocess(source_doc)
print(test)
s = ' '.join(test)
print(s)
print(source_doc[0])


['go', 'green', 'aaa', 'alkaline', 'batteries', 'independently', 'tested', 'verify', 'performance', 'like', 'expensive', 'brands', 'environmentally', 'friendly', 'contains', 'lead', 'cadmium', 'mercury', 'made', 'recycled', 'materials']
go green aaa alkaline batteries independently tested verify performance like expensive brands environmentally friendly contains lead cadmium mercury made recycled materials
G


In [0]:
targetIndex = targetTitles.index("4-port Sunflower/ Mixed Seed Bird Feeder")
# mod_target_preproc = ' '.join(preprocess(mod_target[targetIndex]))
# source_doc_preproc = ' '.join(preprocess(source_doc))
mod_target_preproc = preprocess(mod_target[targetIndex])
source_doc_preproc = preprocess(source_doc)
print(source_doc_preproc)
print(mod_target_preproc)
model.wmdistance(source_doc_preproc, source_doc_preproc)

['go', 'green', 'aaa', 'alkaline', 'batteries', 'independently', 'tested', 'verify', 'performance', 'like', 'expensive', 'brands', 'environmentally', 'friendly', 'contains', 'lead', 'cadmium', 'mercury', 'made', 'recycled', 'materials']
['clean', 'contemporary', 'lines', 'make', 'basic', 'tube', 'feeder', 'attractive', 'addition', 'yard', 'garden', 'quality', 'parts', 'ease', 'use', 'make', 'great', 'value']


0.0

In [0]:
#using WmdSimilarity

wmd_corpus = []
documents = []
documentTitles = []

for i in range(len(target_docs)):
    if target_docs[i] == target_docs[i]: #this is to check if there is an empty string
      # ipdb.set_trace()
      target_docs_preproc = preprocess(target_docs[i])
      if target_docs_preproc != []:
        wmd_corpus.append(target_docs_preproc)
      else:
        wmd_corpus.append([])
    else:
      wmd_corpus.append([])
    documentTitles.append(targetTitles[i])
    documents.append(target_docs[i])
# Initialize WmdSimilarity.
from gensim.similarities import WmdSimilarity
num_best = 10
instance = WmdSimilarity(wmd_corpus, model, num_best=10)



In [0]:
start = time()

query = preprocess(source_doc)

sims = instance[query]  # A query is simply a "look-up" in the similarity class.

In [0]:
# Print the query and the retrieved documents, together with their similarities.
print('Query:')
print(sourceTitle)
for i in range(num_best):
    print()
    print('sim = %.4f' % sims[i][1])
    print(documentTitles[sims[i][0]])

Query:
Go Green AAA Alkaline Battery 8-pack

sim = 1.0000
Go Green AAA Alkaline Battery 8-pack

sim = 0.4990
Duracell Procell Alkaline AAA Batteries (Case of 24)

sim = 0.4986
Duracell Coppertop Alkaline AAA Batteries (Case of 24)

sim = 0.4970
Energizer Industrial Alkaline AAA Batteries (Case of 24)

sim = 0.4948
SmartyKat SweetGreens Cat Grass Kit

sim = 0.4865
Innovera Alkaline AA Batteries (Pack of 8)

sim = 0.4859
Rayovac Ultra Pro Alkaline AA Batteries (Case of 48)

sim = 0.4853
Duracell Procell Alkaline AA Batteries (Case of 24)

sim = 0.4840
Medline AAA Alkaline Battery (Case of 144)

sim = 0.4840
Medline MedCell Alkaline Batteries, AA (Case of 144)


In [0]:
# Showing scores for all poducts above a certain threshold
threshold = 0.4
mod_target = target_docs
# Initialise data to lists.  
#data = [{'Title': 'Score', 'For': 'using', 'geeks': 'list'}, 
#        {'Geeks':10, 'For': 20, 'geeks': 30}]  
## Creates DataFrame.  
#df = pd.DataFrame(data) 

finalListOfDicts = []




print("Title of source product = ", sourceTitle)
print("")
print("Description of source product = ", source_doc)
print("")
print("Items with similarity threshold =  ", threshold, " are below:")

source_doc_preproc = ' '.join(preprocess(source_doc))

for i in range(len(mod_target)):
    if mod_target[i] == mod_target[i]: #this is to check if there is an empty string
        # ipdb.set_trace()
        mod_target_preproc = ' '.join(preprocess(mod_target[i]))
        sim_scores = model.wmdistance(source_doc_preproc, mod_target_preproc)
        if sim_scores != [] and sim_scores < threshold:
            #print(targetTitles[i])
            #print(sim_scores[0]['score'])
            #ipdb.set_trace()
            finalListOfDicts.append({'Title': targetTitles[i], 'score': sim_scores})
            
        
dfFinal = pd.DataFrame(finalListOfDicts)
#dfFinal.sort_values(by=['score'], axis = 1, ascending=False)
dfNew = dfFinal.sort_values(by=['score'], ascending = True)
pd.set_option('display.max_colwidth', -1)
dfNew
#print(sim_scores)

Title of source product =  Go Green AAA Alkaline Battery 8-pack

Description of source product =  Go Green AAA Alkaline Batteries are independently tested to verify performance like the more expensive brands. Environmentally friendly, contains no lead, Cadmium, or Mercury. Made from 100-percecnt recycled materials.

Items with similarity threshold =   0.4  are below:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,Title,score
0,Go Green AAA Alkaline Battery 8-pack,0.000000
1,4-port Sunflower/Mixed Seed Bird Feeder,0.381609
2,4-port Sunflower/ Mixed Seed Bird Feeder,0.381609
3,White Mark Women's 'Bridget' Dress,0.397252
4,Blendtec Designer Series Wildside Blender (Refurbished),0.397945
